## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb　←いまここ
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

In [1]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import re

## 研究機関を抽出する関数

採択時点の研究機関を知りたい。研究機関の情報は、summaryとgrantlistにある。summaryの研究機関は課題終了時の情報になっており、grantlistには年度ごとの研究機関の情報がある。ここではgrantlistから課題ごとに最も古い年度の研究機関データを取得する

In [9]:
def institution(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    institutionlist = []
    for grantAward in tree.iterfind("grantAward"):
        awardnumber = grantAward.get("awardNumber")
        if grantAward.find("grantList/grant[@xml:lang='ja']", nsmap) is None:
            row = [
                awardnumber,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
            ]
            institutionlist.append(row)
        else:
            for grant in grantAward.find("grantList").iterfind("grant[@xml:lang='ja']", nsmap):
                fiscalyear = grant.get("fiscalYear")
                grant_sequence = grant.get("sequence")
                if grant.find("institution") is None:
                    row = [
                        awardnumber,
                        fiscalyear,
                        grant_sequence,
                        np.NaN,
                        np.NaN,
                        np.NaN,
                        np.NaN,
                        np.NaN,
                    ]
                    institutionlist.append(row)

                else:
                    for institution in grant.iterfind("institution"):
                        institution_sequence = institution.get("sequence")
                        institution_niicode = institution.get("niiCode")
                        institution_mextcode = institution.get("mextCode")
                        institution_jspscode = institution.get("jspsCode")
                        institution = institution.text

                        row = [
                            awardnumber,
                            fiscalyear,
                            grant_sequence,
                            institution_sequence,
                            institution_niicode,
                            institution_mextcode,
                            institution_jspscode,
                            institution,
                        ]
                        institutionlist.append(row)
        

    df = pd.DataFrame(institutionlist)
    df.columns = [
        'awardnumber',
        'fiscalyear',
        'grant_sequence',
        'institution_sequence',
        'institution_niicode',
        'institution_mextcode',
        'institution_jspscode',
        'institution',
    ]
    
    pickledfile = 'pickledDF_institution_from_grantlist/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
    df.to_pickle(pickledfile)

フォルダをいったんきれいにする関数を定義して、実行する

In [10]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_institution_from_grantlist')

関数を実行して、XMLファイルごとにパースして、pickleして保存する

In [11]:
for xmlfile in tqdm(glob.glob('xml/201*.xml')):
    institution(xmlfile)

保存したpickleを読み込んで、データフレームを全部連結する

In [14]:
columns = [
    'awardnumber',
    'fiscalyear',
    'grant_sequence',
    'institution_sequence',
    'institution_niicode',
    'institution_mextcode',
    'institution_jspscode',
    'institution',
]
df = pd.DataFrame(columns=columns)

for dump in tqdm(glob.glob('pickledDF_institution_from_grantlist//*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

連結したデータフレームをいったんpickleして保存

In [15]:
df.to_pickle('beforeCleaning/parse_institution_from_grantlist.dump')

### データクリーニング

pickleからデータフレームを復元する

In [16]:
with open('beforeCleaning/parse_institution_from_grantlist.dump', mode='rb') as f:
    df = pickle.load(f)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 545653 entries, 0 to 499
Data columns (total 8 columns):
awardnumber             545653 non-null object
fiscalyear              545653 non-null object
grant_sequence          545653 non-null object
institution_sequence    544944 non-null object
institution_niicode     541157 non-null object
institution_mextcode    535175 non-null object
institution_jspscode    519545 non-null object
institution             544944 non-null object
dtypes: object(8)
memory usage: 37.5+ MB


nullを0で置換する

In [18]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 545653 entries, 0 to 499
Data columns (total 8 columns):
awardnumber             545653 non-null object
fiscalyear              545653 non-null object
grant_sequence          545653 non-null object
institution_sequence    545653 non-null object
institution_niicode     545653 non-null object
institution_mextcode    545653 non-null object
institution_jspscode    545653 non-null object
institution             545653 non-null object
dtypes: object(8)
memory usage: 37.5+ MB


intに指定

In [19]:
df = df.astype({
    'fiscalyear': np.int64,
    'grant_sequence': np.int64,
    'institution_sequence': np.int64,
    'institution_niicode': np.int64,
    'institution_mextcode': np.int64,
    'institution_jspscode': np.int64,    
})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 545653 entries, 0 to 499
Data columns (total 8 columns):
awardnumber             545653 non-null object
fiscalyear              545653 non-null int64
grant_sequence          545653 non-null int64
institution_sequence    545653 non-null int64
institution_niicode     545653 non-null int64
institution_mextcode    545653 non-null int64
institution_jspscode    545653 non-null int64
institution             545653 non-null object
dtypes: int64(6), object(2)
memory usage: 37.5+ MB


awardnumberごとにfiscalyearが最小の行を取得する。

In [20]:
oldest = df.groupby('awardnumber')['fiscalyear'].min().reset_index()
oldest

,awardnumber,fiscalyear
0,08F08045,2008
1,08J03857,2008
2,09J02906,2009
3,09J06155,2009
4,09J06174,2009
5,09J08442,2009
6,09J10929,2009
7,09J55242,2009
8,10F00002,2010
9,10F00003,2010


institutionを連結する

In [21]:
df = pd.merge(oldest, df, on=['awardnumber', 'fiscalyear'])
df

,awardnumber,fiscalyear,grant_sequence,institution_sequence,institution_niicode,institution_mextcode,institution_jspscode,institution
0,08F08045,2008,1,1,17401,17401,17401,熊本大学
1,08J03857,2008,1,1,14401,14401,14401,大阪大学
2,09J02906,2009,1,1,14301,14301,14301,京都大学
3,09J06155,2009,1,1,11301,11301,11301,東北大学
4,09J06174,2009,1,1,12601,12601,12601,東京大学
5,09J08442,2009,1,1,13901,13901,13901,名古屋大学
6,09J10929,2009,1,1,13201,13201,13201,富山大学
7,09J55242,2009,1,1,13601,13601,13601,信州大学
8,10F00002,2010,1,1,11301,11301,11301,東北大学
9,10F00003,2010,1,1,12606,12606,12606,東京芸術大学


重複したデータがあるかどうか

In [22]:
df.duplicated().any()

False

重複はなかった（あったら何かが違う）。

awardnumberがユニークかどうか確認するために、行数255813とawardnumberのユニークな値の数が一致するかどうか確認する。

In [23]:
df['awardnumber'].nunique(dropna=False)

258813

awardnumberはユニークだったので（ユニークじゃなかったら何かが違う）、インデックスにする。

In [24]:
df = df.set_index('awardnumber')
df

,fiscalyear,grant_sequence,institution_sequence,institution_niicode,institution_mextcode,institution_jspscode,institution
awardnumber,,,,,,,
08F08045,2008,1,1,17401,17401,17401,熊本大学
08J03857,2008,1,1,14401,14401,14401,大阪大学
09J02906,2009,1,1,14301,14301,14301,京都大学
09J06155,2009,1,1,11301,11301,11301,東北大学
09J06174,2009,1,1,12601,12601,12601,東京大学
09J08442,2009,1,1,13901,13901,13901,名古屋大学
09J10929,2009,1,1,13201,13201,13201,富山大学
09J55242,2009,1,1,13601,13601,13601,信州大学
10F00002,2010,1,1,11301,11301,11301,東北大学


grantsequenceなど、各列のデータの様子を概観する

In [25]:
pd.crosstab(df['grant_sequence'], df['fiscalyear'], margins=True)

fiscalyear,2004,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
grant_sequence,,,,,,,,,,,,,
1,1,10,43,24433,30031,28824,29432,29677,30708,30917,28984,25753,258813
All,1,10,43,24433,30031,28824,29432,29677,30708,30917,28984,25753,258813


In [26]:
pd.crosstab(df['institution_sequence'], df['fiscalyear'], margins=True)

fiscalyear,2004,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
institution_sequence,,,,,,,,,,,,,
0,0,0,0,40,26,23,26,27,21,29,28,6,226
1,1,10,43,24393,30005,28801,29406,29650,30687,30888,28956,25747,258587
All,1,10,43,24433,30031,28824,29432,29677,30708,30917,28984,25753,258813


In [27]:
pd.crosstab(df['institution_niicode'], df['fiscalyear'], margins=True)

fiscalyear,2004,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
institution_niicode,,,,,,,,,,,,,
0,0,0,1,276,286,538,465,245,240,243,231,129,2654
10101,0,0,1,597,835,787,751,751,698,751,646,575,6392
10102,0,0,0,28,31,36,37,33,40,44,28,27,304
10103,0,0,0,16,26,26,27,25,26,33,16,17,212
10104,0,0,0,7,14,12,10,15,10,10,16,11,105
10105,0,0,0,19,18,18,27,26,24,20,22,29,203
10106,0,0,0,10,17,27,21,15,30,22,14,22,178
10107,0,0,0,35,49,46,35,51,68,49,59,54,446
11101,0,0,2,90,138,116,126,128,119,116,129,123,1087


In [28]:
pd.crosstab(df['institution_mextcode'], df['fiscalyear'], margins=True)

fiscalyear,2004,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
institution_mextcode,,,,,,,,,,,,,
0,0,0,0,258,309,1566,2000,238,240,243,215,129,5198
10101,0,0,1,597,835,787,751,751,698,751,646,575,6392
10102,0,0,0,28,31,36,37,33,40,44,28,27,304
10103,0,0,0,16,26,26,27,25,26,33,16,17,212
10104,0,0,0,7,14,12,10,15,10,10,16,11,105
10105,0,0,0,19,18,18,27,26,24,20,22,29,203
10106,0,0,0,10,17,27,21,15,30,22,14,22,178
10107,0,0,0,35,49,46,35,51,68,49,59,54,446
11101,0,0,2,90,138,116,126,128,119,116,129,123,1087


In [29]:
pd.crosstab(df['institution_jspscode'], df['fiscalyear'], margins=True)

fiscalyear,2004,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
institution_jspscode,,,,,,,,,,,,,
0,0,0,7,1763,1972,655,334,2022,1783,1699,1621,1410,13266
10101,0,0,1,597,835,787,751,751,698,751,646,575,6392
10102,0,0,0,28,31,36,37,33,40,44,28,27,304
10103,0,0,0,16,26,26,27,25,26,33,16,17,212
10104,0,0,0,7,14,12,10,15,10,10,16,11,105
10105,0,0,0,19,18,18,27,26,24,20,22,29,203
10106,0,0,0,10,17,27,21,15,30,22,14,22,178
10107,0,0,0,35,49,46,35,51,68,49,59,54,446
11101,0,0,2,90,138,116,126,128,119,116,129,123,1087


niicode、mextcode、jspscodeで比較して、コード0が最も少ないのはniicodeだった。これからniicodeを使うことにする。

研究機関名institutionとniicodeの対応状況を見る。たとえば京大だと…

In [30]:
kyodai = df[df['institution_niicode'] == 14301]
pd.crosstab(kyodai['institution'], kyodai['institution_niicode'])

institution_niicode,14301
institution,
äº¬éƒ½å¤§å­¦,78
京都大学,12446
京都大学医学研究科,1
京都大学医学部附属病院,2
国立大学法人京都大学,1


研究機関名institutionとniicodeの対応状況を見る。たとえば東大だと…

In [31]:
todai = df[df['institution_niicode'] == 12601]
pd.crosstab(todai['institution'], todai['institution_niicode'])

institution_niicode,12601
institution,
æ�±äº¬å¤§å­¦,107
東京大学,17318
東京大学大学院 工学系研究科,1
秋田県立大学,1
鹿児島大学,1


In [32]:
df.query('institution_niicode == 12601 & institution == "秋田県立大学"')

,fiscalyear,grant_sequence,institution_sequence,institution_niicode,institution_mextcode,institution_jspscode,institution
awardnumber,,,,,,,
22710135,2010,1,1,12601,12601,12601,秋田県立大学


In [33]:
df.query('institution_niicode == 12601 & institution == "鹿児島大学"')

,fiscalyear,grant_sequence,institution_sequence,institution_niicode,institution_mextcode,institution_jspscode,institution
awardnumber,,,,,,,
22740005,2010,1,1,12601,12601,12601,鹿児島大学


数件違うけど、調べてみたら、内定の前後で異動している人のようだ。niicodeをベースにすることにする。

ここまでで中間データをいったんpickleして保存

In [34]:
df.to_pickle('afterCleaning/parse_institution_from_grantlist.dump')

ひとまず終了。

次は、parse_grantaward_member_from_summary.ipynbで内定時の研究代表者を取得する。